In [0]:
%sql
DROP TABLE dev.bronze.emp;

In [0]:
%sql
    
CREATE TABLE dev.bronze.emp
(
  emp_id INT,
  emp_name STRING,
  dept_code STRING,
  salary INT,
  is_active string
)
USING delta;

In [0]:
%sql
INSERT INTO TABLE dev.bronze.emp VALUES
(1001,'John','D101',15000,'Y'),
(1002,'Mary','D101',13000,'Y'),
(1003,'Mike','D102',10000,'Y'),
(1004,'Jane','D103',10000,'Y'),
(1005,'Mary','D103',10000,'N');


num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
-- EMP DATA
select * from dev.bronze.emp

emp_id,emp_name,dept_code,salary,is_active
1001,John,D101,15000,Y
1002,Mary,D101,13000,Y
1003,Mike,D102,10000,Y
1004,Jane,D103,10000,Y
1005,Mary,D103,10000,N


In [0]:
%sql
select current_user()

current_user()
jampala.teja_gmail.com#ext#@jampalatejagmail.onmicrosoft.com


In [0]:
%sql
CREATE OR REPLACE FUNCTION dev.bronze.tax_sql(sal DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN sal * 0.3

In [0]:
%sql
select dev.bronze.tax_sql(10000)

dev.bronze.tax_sql(10000)
3000.0


In [0]:
%sql
select *,dev.bronze.tax_sql(salary) as tax from dev.bronze.emp

emp_id,emp_name,dept_code,salary,is_active,tax
1001,John,D101,15000,Y,4500.0
1002,Mary,D101,13000,Y,3900.0
1003,Mike,D102,10000,Y,3000.0
1004,Jane,D103,10000,Y,3000.0
1005,Mary,D103,10000,N,3000.0


In [0]:
from pyspark.sql.functions import expr

df = spark.read.table("dev.bronze.emp")
df.withColumn("tax", expr("dev.bronze.tax_sql(salary)")).display()

emp_id,emp_name,dept_code,salary,is_active,tax
1001,John,D101,15000,Y,4500.0
1002,Mary,D101,13000,Y,3900.0
1003,Mike,D102,10000,Y,3000.0
1004,Jane,D103,10000,Y,3000.0
1005,Mary,D103,10000,N,3000.0


In [0]:
%sql
-- SCALER FUNCTION FOR API
CREATE OR REPLACE FUNCTION dev.bronze.fruit_nutrition(fruit_name STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  import requests
  api_url = f"https://fruityvice.com/api/fruit/{fruit_name}"
  response = requests.get(api_url)
  data = response.json()
  return str(data.get('nutritions','NA'))
$$
;

In [0]:
%sql
select dev.bronze.fruit_nutrition('apple')

dev.bronze.fruit_nutrition(apple)
"{'calories': 52, 'fat': 0.4, 'sugar': 10.3, 'carbohydrates': 11.4, 'protein': 0.3}"


In [0]:
%sql
-- CREATE TABLE FUNCTION TO GET ALL EMP FROM A PARTICULAR DEPT
CREATE OR REPLACE FUNCTION dev.bronze.get_emp(dept STRING)
RETURNS TABLE (emp_id INT, emp_name STRING, dept_code STRING, salary INT, is_active STRING)
LANGUAGE SQL
RETURN(
  SELECT * FROM dev.bronze.emp WHERE dept_code = dept
)
;
    
--select * from dev.bronze.get_emp('D101')

In [0]:
%sql
-- VALIDATE FUNCTION
select * from dev.bronze.get_emp('D101')


emp_id,emp_name,dept_code,salary,is_active
1001,John,D101,15000,Y
1002,Mary,D101,13000,Y
